In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = (12,12)
mpl.rcParams['axes.grid'] = False

from util import imshow, load_img, save_img, apply_lum, match_lum
from model import StyleTransferModel
from losses import content_loss
from train import train

# https://www.positive.news/wp-content/uploads/2019/03/feat-1800x0-c-center.jpg
content_path = tf.keras.utils.get_file('forest.jpg','file:///home/jupyter/pictures/forest.jpg')

content_img = load_img(content_path, max_dim=512)
imshow(content_img, 'Content Image')

plt.show()

In [ ]:
# reconstruct content, for every layer
content_layers = ['block1_conv1',
                  'block2_conv1',
                  'block3_conv1', 
                  'block4_conv1', 
                  'block5_conv1']

for content_layer in content_layers:
    extractor = StyleTransferModel(['block1_conv1'], [content_layer])

    content_targets = extractor(content_img)['content']
    content_weights = [1.0]

    # initialize the gradients with random noise
    initial_gradients = tf.Variable(tf.random.uniform(content_img.shape))

    def loss_func(image):
        outputs = extractor(image)
        loss = content_loss(outputs['content'], content_targets, content_weights)
        return loss

    result = train(loss_func, extractor, initial_gradients, epochs=20)

    save_img(result[0], 'content_{}.png'.format(content_layer))

In [ ]:
for i, content_layer in enumerate(content_layers):
    plt.subplot(3, 2, i+1)
    img = load_img('output/content_{}.png'.format(content_layer), max_dim=512)
    imshow(result, 'Content Layer {}'.format(content_layer))